<a href="https://colab.research.google.com/github/Arvore420/Dados-Financeiros-Merge/blob/main/Merge_concat_duas_tabelas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# obtenção dos dados históricos das cotações da carteira de 01/08/2022 à 01/08/2023
# Importando a biblioteca
import yfinance as yf

# Definindo a carteira de ações
carteira_yf = ['ABEV3.SA', 'B3SA3.SA', 'ELET3.SA', 'GGBR4.SA', 'ITSA4.SA',
               'PETR4.SA', 'RENT3.SA', 'SUZB3.SA', 'VALE3.SA', 'WEGE3.SA']

# Carregando os dados da carteira
df = yf.download(carteira_yf, start="2022-08-01", end="2023-08-01")

# Passando os ativos para o multindex do df
cotacoes = df.stack(level=1)

# Resetando os índices e renomenado a coluna dos ativos
cotacoes = cotacoes.reset_index().rename(columns={"Ticker": "Ativo"})

# Organizando o df
cotacoes = cotacoes[["Date", "Open", "High", "Low", "Close", "Ativo"]]

del carteira_yf, df

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  10 of 10 completed
<ipython-input-1-c98e2e6d1140>:13: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  cotacoes = df.stack(level=1)


In [6]:
cotacoes

Price,Date,Open,High,Low,Close,Ativo
0,2022-08-01,13.034962,13.061278,12.657773,12.833210,ABEV3.SA
1,2022-08-01,10.103378,10.195060,9.819163,9.965855,B3SA3.SA
2,2022-08-01,43.906293,44.951225,43.772082,44.577351,ELET3.SA
3,2022-08-01,15.786208,15.786208,15.061952,15.170911,GGBR4.SA
4,2022-08-01,5.592136,5.611530,5.462838,5.521022,ITSA4.SA
...,...,...,...,...,...,...
2495,2023-07-31,22.920215,23.563191,22.920215,23.532934,PETR4.SA
2496,2023-07-31,63.462296,64.049205,62.894317,63.632687,RENT3.SA
2497,2023-07-31,44.844430,45.998898,44.797116,45.487904,SUZB3.SA
2498,2023-07-31,57.575336,58.997786,57.439861,58.557507,VALE3.SA


In [ ]:
!pip install fundamentus yfinance

In [4]:
# Importando as bibliotecas
import fundamentus
import pandas as pd

# Definindo a carteira de ações
carteira_fund = ["ABEV3", "B3SA3", "ELET3", "GGBR4", "ITSA4",
                 "PETR4", "RENT3", "SUZB3", "VALE3", "WEGE3"]

# Criando um df com algumas infos da carteira
ind = pd.concat([fundamentus.get_papel(papel)[['Setor', 'Cotacao', 'Min_52_sem', 'Max_52_sem', 'Valor_de_mercado',
                                            'Nro_Acoes', 'Patrim_Liq','Receita_Liquida_12m','Receita_Liquida_3m',
                                            'Lucro_Liquido_12m', 'Lucro_Liquido_3m']] for papel in carteira_fund])

# Passando o ticker para uma coluna
ind = ind.reset_index()
ind.rename(columns = {'index':'Ativo'}, inplace=True)

# Alterando colunas object para numeric
colunas = ['Cotacao', 'Min_52_sem', 'Max_52_sem', 'Valor_de_mercado', 'Nro_Acoes', 'Patrim_Liq',
           'Receita_Liquida_12m', 'Receita_Liquida_3m', 'Lucro_Liquido_12m', 'Lucro_Liquido_3m']
ind[colunas] = ind[colunas].apply(pd.to_numeric, errors='coerce', axis=1)

# Criando um novo df com alguns indicadores da carteira
ind_2 = fundamentus.get_resultado_raw().reset_index()
ind_2 = ind_2.query("papel in @carteira_fund")
ind_2 = ind_2[['papel','P/L', 'Div.Yield','P/VP','ROE']].reset_index(drop=True)
ind_2.rename(columns={'papel': 'Ativo','Div.Yield':'DY'}, inplace= True)

# Concatenando os dfs em um só com as infos e indicadores
indicadores = pd.merge(ind, ind_2, on="Ativo")

# Criando uma coluna para LPA (Lucro por Ação) e VPA (Valor Patrimonial por ação)
# para calcular a fórmula de Graham  Valor intrínseco de uma ação (VI = √22,5 x LPA x VPA)
indicadores["LPA"] = (indicadores["Lucro_Liquido_12m"] / indicadores ["Nro_Acoes"]).round(2)
indicadores["VPA"] = (indicadores["Patrim_Liq"] / indicadores ["Nro_Acoes"]).round(2)

del ind, ind_2, carteira_fund, colunas

/usr/local/lib/python3.11/dist-packages/fundamentus/detalhes.py:232: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables_html = pd.read_html(content.text, decimal=",", thousands='.')
/usr/local/lib/python3.11/dist-packages/fundamentus/detalhes.py:232: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables_html = pd.read_html(content.text, decimal=",", thousands='.')
/usr/local/lib/python3.11/dist-packages/fundamentus/detalhes.py:232: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables_html = pd.read_html(content.text, decimal=",", thousands='.')
/usr/local/lib/python3.11/dist-packages/fundamentus/detalhes.py:232: Futu

In [5]:
indicadores

,Ativo,Setor,Cotacao,Min_52_sem,Max_52_sem,Valor_de_mercado,Nro_Acoes,Patrim_Liq,Receita_Liquida_12m,Receita_Liquida_3m,Lucro_Liquido_12m,Lucro_Liquido_3m,P/L,DY,P/VP,ROE,LPA,VPA
0,ABEV3,Bebidas,13.73,10.45,13.73,2.163530e+11,1.575770e+10,9.864210e+10,8.945270e+10,2.703540e+10,1.443720e+10,4.880380e+09,14.99,0.0579,2.19,0.1464,0.92,6.26
1,B3SA3,Serviços Financeiros Diversos,12.56,9.10,12.87,6.815680e+10,5.426500e+09,1.837080e+10,1.057270e+10,2.667800e+09,4.576580e+09,1.178460e+09,14.89,0.0243,3.71,0.2491,0.84,3.39
2,ELET3,Energia Elétrica,41.93,33.21,42.21,9.673670e+10,2.307100e+09,1.218630e+11,4.018160e+10,1.202510e+10,1.037810e+10,1.114490e+09,9.32,0.0302,0.79,0.0852,4.50,52.82
3,GGBR4,Siderurgia e Metalurgia,17.14,16.12,21.02,3.518350e+10,2.052710e+09,5.794880e+10,6.702670e+10,1.682250e+10,4.566320e+09,3.160230e+08,7.71,0.0479,0.61,0.0788,2.22,28.23
4,ITSA4,Intermediários Financeiros,9.56,7.96,9.88,1.036740e+11,1.084460e+10,9.044300e+10,8.235000e+09,2.064000e+09,1.477800e+10,3.722000e+09,7.02,0.0927,1.15,0.1634,1.36,8.34
5,PETR4,"Petróleo, Gás e Biocombustíveis",36.80,29.69,38.50,4.743050e+11,1.288870e+10,3.660060e+11,4.908290e+11,1.212680e+11,3.660600e+10,-1.704400e+10,12.96,0.2165,1.30,0.1000,2.84,28.40
6,RENT3,Diversos,33.46,27.60,53.72,3.622450e+10,1.082620e+09,2.633410e+10,3.727160e+10,9.852750e+09,1.813630e+09,8.381040e+08,19.97,0.0470,1.38,0.0689,1.68,24.32
7,SUZB3,Madeira e Papel,54.35,45.39,64.87,6.870480e+10,1.264120e+09,3.228430e+10,4.740330e+10,1.417700e+10,-7.074200e+09,-6.748300e+09,-9.71,0.0371,2.13,-0.2191,-5.60,25.54
8,VALE3,Mineração,57.45,49.30,61.36,2.607660e+11,4.539010e+09,2.067720e+11,2.060050e+11,5.940100e+10,3.159200e+10,-4.677000e+09,8.25,0.0828,1.26,0.1528,6.96,45.55
9,WEGE3,Máquinas e Equipamentos,47.85,36.83,58.11,2.008420e+11,4.197320e+09,2.220420e+10,3.798690e+10,1.082230e+10,6.042590e+09,1.694300e+09,33.24,0.0164,9.05,0.2721,1.44,5.29
